# Test the trained prototype model

In [1]:
import sys
import os

# Go up to project root (from inside training/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
import torch
import numpy as np
from core.utils import process_sample

# Old Model
from core.models.hierarchical_transformer_prototype import HierarchicalTransformer as HierarchicalTransformerPrototype
# Final Model
from core.models.hierarchical_transformer import HierarchicalTransformer


## Create a simple pipeline
This will include:
- using raw video data.
- Processing the raw video data.
- Passing the processed video data into the model.
- Displaying the results.

In [3]:

from core.keypoint_extractor import KeypointExtractorV2

In [4]:
def load_and_extract(path: str) -> np.ndarray:
    extractor = KeypointExtractorV2(r"../../models/mediapipe/pose_landmarker_full.task")
    keypoints = extractor.extract(path)
    
    max_frames = 201
    pad_len = max_frames - len(keypoints)
    if pad_len > 0:
        pad = np.zeros((pad_len, keypoints.shape[1], keypoints.shape[2]))  # Preserve all dimensions
        padded_sample = np.concatenate((keypoints, pad), axis=0)
    else:
        padded_sample = keypoints

    return np.array(padded_sample)
        

In [5]:
def load_and_extract_for_inference(video_path: str, max_frames: int = 200) -> tuple[np.ndarray, np.ndarray]:
    extractor = KeypointExtractorV2(r"../../models/mediapipe/pose_landmarker_full.task")
    keypoints = extractor.extract(video_path) # keypoints will be (actual_len, num_keypoints, coords)
    padded_sample, attention_mask,_ = process_sample(keypoints, max_frames)
    return (padded_sample, attention_mask)

In [6]:
labels_map = {0: "Squats", 1: "Deadlifts", 2: "Shoulder Press"}

In [ ]:
model_final = HierarchicalTransformer(
   num_joints=33,
    num_frames=201,
    d_model=64,
    nhead=2,
    num_spatial_layers=1,
    num_temporal_layers=1,
    num_classes=3,
    dim_feedforward=2048,
    dropout=0.1
)
model_final.load_state_dict(torch.load("../../models/hierarchical_transformer/hierarchical_transformer_f201_d64_h2_s1_t1_do0.1_20250630_0325.pth"))

<All keys matched successfully>

In [10]:
def final_model_inference(video_to_test_path):
    x_np, mask_np = load_and_extract_for_inference(video_to_test_path, max_frames=200)
    
    x_np = x_np[:, :, :3] # get x y z only
    x_tensor = torch.from_numpy(x_np).float().unsqueeze(0) # Add batch dimension
    mask_tensor = torch.from_numpy(mask_np).float().unsqueeze(0) # Add batch dimension for mask

    # Ensure tensors are on the correct device (CPU or GPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_final.to(device)
    x_tensor = x_tensor.to(device)
    mask_tensor = mask_tensor.to(device)

    with torch.no_grad(): # Disable gradient calculation for inference
        # Pass both the input data and the attention mask to the model
        logits = model_final(x_tensor, temporal_mask=mask_tensor)

        # Get the predicted class (highest logit)
        predicted_class_idx = torch.argmax(logits, dim=1).item()
        
        # Get the probabilities (if needed)
        probabilities = torch.softmax(logits, dim=1)[0] # [0] because it's a batch of 1

    # Print results
    print("New Model (V2):")
    print(f"\nVideo: {video_to_test_path}")
    print(f"Predicted Class Index: {predicted_class_idx}")
    print(f"Predicted Exercise: {labels_map.get(predicted_class_idx, 'Unknown')}")

In [23]:

video_to_test_path = '../../data/unseen/squat_neil_2.mp4'
print("--------------------------------------------------------------------------------------")
final_model_inference(video_to_test_path)

--------------------------------------------------------------------------------------
Processing ../../data/unseen/squat_neil_2.mp4: 534x720, 39 frames
Extracted and normalized 39 frames from ../../data/unseen/squat_neil_2.mp4
New Model (V2):

Video: ../../data/unseen/squat_neil_2.mp4
Predicted Class Index: 2
Predicted Exercise: Shoulder Press
